In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torch.nn as nn

## Variable multi-categorica

Es una variable categorica comun, con la diferencia que, la columna en la observación no tiene un solo valor, sino una lista de valores.

Ej.: Las peliculas pertenecen a varios generos. El genero es una variable categorica común, pero las peliculas pueden petenecer mas de un genero. Entonces una observacion de una pelicula podria ser:

- Nombre: Toy Story
- Generos: [Comedia, Fantasticas, Aventura]

## Embedding Bag

* Un Embedding bag permite sumar, promediar(pesado o normal) o quedarnos con el maximo de una lista de embedding vectors.
* Son muy usados cuando tenemos una variable muti-categorica.
* Cuando se necesita utilizar un promedio pesado, el problema es que los pesos no son parametros a aprender, si no que hay que pasarlos. Son parametros fijos :(.
* Lo mejor seria tener un EmbeddingBag que aprenda esos pesos ajustandolos en el proceso de backpropagation ;)

## Weighted Mean Embedding Bag

* A continuación se implementa un EmbeddingBag con promedio pesado, donde los pesos son parametros 
a apender por la capa (Módulo en Pytorch).
* De esta forma, se separa el problema es dos pasos:
  * Una capa embedding común la cual, en base a los indices de las categorias devuelve embedding vectors.
  * Otra capa (**LinearWeightedAVG**) que toma estos vectores, hace el promedio pesado y se queda con un único vector embedding promedio para cada batch.

In [237]:
list_to_tensor = lambda list: torch.stack(list).squeeze(-2)


class ObservationEmbeddingsWeightedMean(nn.Module):
    def __init__(self):
        super(LinearWeightedMean, self).__init__()
        self.weights = None

    def forward(self, batches):
        if self.weights == None:
            embedding_size = batches.shape[-2]
            self.weights = nn.ParameterList([nn.Parameter(torch.randn(1)) for i in range(embedding_size)])

        output_batches = []
        for batch in batches:
            output_batch = []
            for observation in batch:
                weighted_embs = list_to_tensor([emb * self.weights[idx] for idx, emb in enumerate(observation)])
                emb_mean = torch.mean(weighted_embs, dim=0)
                output_batch.append(emb_mean)

            output_batches.append(list_to_tensor(output_batch))

        return list_to_tensor(output_batches)


class WeightedMeanEmbeddingBag(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super(WeightedMeanEmbeddingBag, self).__init__()
        self._emb = embedding = nn.Embedding(num_embeddings, embedding_dim)
        self._avg = LinearWeightedMean()
 
    def forward(self, x): return torch.mean(self._emb(x), dim=-2)

## Ejemplo

* Cada observación es una lista de categorias de una variable categorica codificadas en numeros.
* La variable categorica tiene 3 posibles valores, excluyentes en cada posición de la lista de valores. Por ej.: una pelicula no puede tener dos veces el genero comedia.
* Cada observacion es una lista de tamaño 3, por que una pelicula podria tener todos los generos posibles (3 en este ejemplo).
* Algunas peliculas pueden tener menos generos que el total. Los que faltantes quedan en cero.

In [90]:
embedding = nn.Embedding(
    num_embeddings=4, # La opcion sin genero es un valor mas de la categorica.
    embedding_dim=2
)
embedding.weight

Parameter containing:
tensor([[ 6.3600e-01, -1.6904e+00],
        [ 1.2728e+00,  6.3747e-01],
        [ 2.0974e+00, -1.5614e+00],
        [-4.9120e-01,  8.4095e-04]], requires_grad=True)

In [91]:
input_ = torch.LongTensor([
    [ 
        [1, 2, 3], # La pelicula 1 tiene los generos 1,2 y 3.
        [3, 0, 0]  # La pelicula 2 tiene el generos 3 solamente.
    ],
    [ 
        [1, 2, 0], 
        [2, 0, 0]
    ]
])

input_.size()

torch.Size([2, 2, 3])

Tenemos un 2 lotes de 2 observaciones cada uno:

In [92]:
input_

tensor([[[1, 2, 3],
         [3, 0, 0]],

        [[1, 2, 0],
         [2, 0, 0]]])

In [93]:
embed = embedding(input_)

In [94]:
embed.size()

torch.Size([2, 2, 3, 2])

In [95]:
embed

tensor([[[[ 1.2728e+00,  6.3747e-01],
          [ 2.0974e+00, -1.5614e+00],
          [-4.9120e-01,  8.4095e-04]],

         [[-4.9120e-01,  8.4095e-04],
          [ 6.3600e-01, -1.6904e+00],
          [ 6.3600e-01, -1.6904e+00]]],


        [[[ 1.2728e+00,  6.3747e-01],
          [ 2.0974e+00, -1.5614e+00],
          [ 6.3600e-01, -1.6904e+00]],

         [[ 2.0974e+00, -1.5614e+00],
          [ 6.3600e-01, -1.6904e+00],
          [ 6.3600e-01, -1.6904e+00]]]], grad_fn=<EmbeddingBackward0>)

In [96]:
embed.size()

torch.Size([2, 2, 3, 2])

In [239]:
avg = LinearWeightedMean()

In [240]:
avg(embed)

tensor([[[ 1.4879, -0.8804],
         [ 0.1660, -0.5460]],

        [[ 1.1796, -0.4179],
         [ 0.3741, -0.6716]]], grad_fn=<SqueezeBackward1>)

In [242]:
avg.weights[0], avg.weights[1], avg.weights[2]

(
    Parameter containing:
tensor([0.2412], requires_grad=True),
    Parameter containing:
tensor([1.7897], requires_grad=True),
    Parameter containing:
tensor([-0.8205], requires_grad=True)
)

In [214]:
cv = nn.Conv1d(in_channels=embed.size()[-2], out_channels=1, kernel_size=1)

In [86]:
cv

Conv1d(3, 1, kernel_size=(1,), stride=(1,))

In [71]:
cv.weight, cv.bias

(
    Parameter containing:
tensor([[[-0.0121],
         [-0.4894],
         [-0.4531]]], requires_grad=True),
    Parameter containing:
tensor([0.3587], requires_grad=True)
)

In [118]:
torch.mean(embed[0][0], dim=0)

tensor([ 0.9597, -0.3077], grad_fn=<MeanBackward1>)

In [117]:
torch.mean(embed[0][0], dim=0)

tensor([ 0.9597, -0.3077], grad_fn=<MeanBackward1>)

In [70]:
out = cv(embed[0][0])
out

tensor([[-0.8034,  0.4144]], grad_fn=<SqueezeBackward1>)

In [13]:
out.size()

torch.Size([2, 2, 2])

In [18]:
wmean = WeightedMeanEmbeddingBag(num_embeddings=4, embedding_dim=2)

In [19]:
out = wmean(input_)
out

tensor([[[0.3720, 0.8317],
         [0.2079, 0.2181]],

        [[0.3356, 0.3721],
         [0.1486, 0.2507]]], grad_fn=<SqueezeBackward1>)

In [20]:
wmean

WeightedMeanEmbeddingBag(
  (_emb): Embedding(4, 2)
  (_avg): LinearWeightedMean(
    (_avg): Conv1d(3, 1, kernel_size=(1,), stride=(1,))
  )
)

In [21]:
out.size()

torch.Size([2, 2, 2])

### References

* https://stackoverflow.com/questions/58568400/weighted-summation-of-embeddings-in-pytorch